In [20]:
import tensorflow as tf
import keras_tuner
import numpy as np


import pandas as pd
import openpyxl # Needed for reading excel
import pathlib

import decomposition
import models
import data
from metrics import smape
from windower import WindowGenerator
import hp_training
import results

In [119]:
cwd = pathlib.Path.cwd()

code_directory = cwd.parents[1]
gonem_directory = code_directory / "notebooks" / "Gonem"
data_directory = code_directory / "data"
hp_directory = code_directory / "hp"
scenario_directory = code_directory / "scenarios"
model_directory = code_directory / 'models'

model = 'ED' # SS, ARF or ED
product = 'wheat' # maize, sunflower or wheat 

data_type = 'in_sample' # in_sample or out_sample
scenario = 0

model_path =  model_directory / f"{model}_{product}"
checkpoint_path = model_path / f"{model}_{product}"

results_path = model_directory / f"{model}_{product}"

results_path

scenario_files = []
for path in pathlib.Path(scenario_directory).iterdir():
    if path.is_file():
        scenario_files.append(path.name)

In [120]:
if scenario:
    data_type = f"{scenario_files[scenario-1]}"
    print(data_type)
    df = data.get_data(scenario_directory / scenario_files[scenario-1])
else:
    df = data.get_data(directory_path=data_directory, product=product)


df = df.iloc[:-2]
df.describe()

AVG_TAVG                                                     \
PARTNER_Labels     Belgium      France     Germany     Romania United Kingdom   
count           214.000000  214.000000  214.000000  214.000000     214.000000   
mean            111.302520  125.494113  104.143755  105.397067      96.544608   
std              57.100073   55.456797   65.178828   81.500731      39.315210   
min              -0.733333   20.168095  -29.860742  -65.129397       7.340256   
25%              63.525974   77.127932   48.280935   29.836943      61.896038   
50%             104.345960  121.548459  102.466712  104.869216      96.188597   
75%             163.724856  177.640994  163.845330  182.880379     131.974731   
max             239.666667  231.747995  229.864177  243.079179     178.133333   

                  MAX_TMAX                                                     \
PARTNER_Labels     Belgium      France     Germany     Romania United Kingdom   
count           214.000000  214.000000  214.000000  214.000000     214.000000   
mean            215.733645  225.531663  218.804582  224.761987     168.507655   
std              79.022190   67.153277   81.278888   81.711107      50.004673   
min              59.000000  107.622222   36.388889   47.000000      74.500000   
25%             147.500000  163.656487  140.181174  157.733333     124.583333   
50%             223.000000  228.556851  226.650735  229.750000     164.437500   
75%             281.000000  284.822581  287.334967  300.500000     208.409091   
max             397.000000  362.382979  371.823529  367.750000     300.562500   

                ... renewable_energy_consumption_perc_of_total              \
PARTNER_Labels  ...                                    Belgium      France   
count           ...                                 214.000000  214.000000   
mean            ...                                   7.672173   12.883762   
std             ...                                   2.698506    2.273139   
min             ...                                   2.460000    8.520000   
25%             ...                                   5.438125   11.155000   
50%             ...                                   9.016250   13.286667   
75%             ...                                  10.210000   15.310000   
max             ...                                  10.660000   15.530000   

                                                      unemployment_total  \
PARTNER_Labels     Germany     Romania United Kingdom            Belgium   
count           214.000000  214.000000     214.000000         214.000000   
mean             13.639136   22.297593       7.120444           7.272757   
std               2.851412    2.019432       3.915944           0.994389   
min               7.280000   17.390000       1.350000           5.360000   
25%              11.109375   21.416250       3.506875           6.260000   
50%              13.971250   23.290833       7.192500           7.460000   
75%              16.448125   23.540000      11.380625           8.149583   
max              17.170000   24.400000      12.240000           8.520000   

                                                                   
PARTNER_Labels      France     Germany     Romania United Kingdom  
count           214.000000  214.000000  214.000000     214.000000  
mean              8.969860    5.587944    6.091075       5.505070  
std               0.867082    2.216610    0.996584       1.547705  
min               7.390000    3.140000    3.910000       3.740000  
25%               8.086250    3.646250    5.508333       3.918750  
50%               9.068750    5.011250    6.412083       5.278750  
75%               9.789167    7.403125    6.926250       7.140625  
max              10.350000   11.170000    7.270000       8.040000  

[8 rows x 63 columns]

In [121]:
label_columns = ['price']
label_columns = df.columns[df.columns.get_level_values(0).isin(label_columns)].tolist()
label_columns

[('price', 'Belgium'),
 ('price', 'France'),
 ('price', 'Germany'),
 ('price', 'Global'),
 ('price', 'Romania'),
 ('price', 'United Kingdom')]

In [122]:
stl = decomposition.STLDecomposer(labels=label_columns, period=12)
log = decomposition.Logger(labels=label_columns)
std = decomposition.Standardizer()

preproc = decomposition.Processor().add(stl).add(log).add(std)

In [123]:
width = 24
label_width = 6
shift = 6

if data_type == 'in_sample':
    test_begin = None
else:
    test_begin = 0.
    
window = WindowGenerator(input_width=width, label_width=label_width, shift=shift, data=df, 
                    # train_begin=0, train_end=.9, val_begin=None, val_end=.96,
                    train_begin=0., train_end=.97, val_begin=None, val_end=None,
                    # train_begin=0, train_end=.5, val_begin=None, val_end=.8,
                    test_begin=test_begin, test_end=1., connect=True, remove_labels=True, label_columns=label_columns)
window.preprocess(preproc)
window

Total window size: 30
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
Label indices: [24 25 26 27 28 29]
Label column name(s): [('price', 'Belgium'), ('price', 'France'), ('price', 'Germany'), ('price', 'Global'), ('price', 'Romania'), ('price', 'United Kingdom')]

In [124]:
label_std = decomposition.Standardizer(mean=std.mean[window.label_columns], std=std.std[window.label_columns])
label_log = decomposition.Logger(label_indices=range(len(window.label_columns)))
postproc = decomposition.Processor().add(label_std).add(label_log)
window.add_label_postprocess(postproc)

In [125]:
tuner = hp_training.get_tuner(model, hp_directory, window)

INFO:tensorflow:Reloading Tuner from /code/hp/ED/tuner0.json


In [126]:
for example_inputs, example_labels in window.train.take(1):
    print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
    print(f'Labels shape (batch, time, features): {example_labels.shape}')
    output_features = example_labels.shape[-1]

Inputs shape (batch, time, features): (32, 24, 75)
Labels shape (batch, time, features): (32, 6, 6)


# Here we train the best hp model and give it a final training

In [112]:
# hp_training.run(tuner, window)

In [113]:
best_hps = tuner.get_best_hyperparameters()[0]
best_hps.values

{'encoder_units': 256,
 'encoder_layers': 2,
 'decoder_units': 224,
 'decoder_layers': 4,
 'dense_units': 320,
 'dense_layers': 5,
 'heads': 4,
 'dropout': 0.0656398128830259,
 'key_dim': 32,
 'l1': 5.169932032477925e-07,
 'l2': 0.00010441578807915571,
 'learning_rate': 0.00024152676688158866,
 'tuner/epochs': 200,
 'tuner/initial_epoch': 67,
 'tuner/bracket': 3,
 'tuner/round': 3,
 'tuner/trial_id': '0203'}

In [114]:
hp_training.final_train(tuner, window, checkpoint_path)

Epoch 1/200
6/6 - 12s - loss: 189.2620 - mae: 0.7683 - mse: 0.9404 - mape: 102.5098 - smape: 187.1148 - val_loss: 168.5220 - val_mae: 0.7652 - val_mse: 0.9552 - val_mape: 120.3482 - val_smape: 167.1693 - 12s/epoch - 2s/step
Epoch 2/200
6/6 - 1s - loss: 158.0757 - mae: 0.7737 - mse: 0.9940 - mape: 138.4746 - smape: 156.5593 - val_loss: 139.9650 - val_mae: 0.8332 - val_mse: 1.1891 - val_mape: 219.7452 - val_smape: 139.6573 - 795ms/epoch - 132ms/step
Epoch 3/200
6/6 - 1s - loss: 145.9551 - mae: 0.9981 - mse: 1.5213 - mape: 283.6198 - smape: 146.4142 - val_loss: 137.8200 - val_mae: 1.1261 - val_mse: 1.9731 - val_mape: 401.7635 - val_smape: 138.6097 - 870ms/epoch - 145ms/step
Epoch 4/200
6/6 - 1s - loss: 140.9851 - mae: 1.1364 - mse: 2.0299 - mape: 403.2751 - smape: 141.5529 - val_loss: 132.1713 - val_mae: 0.9902 - val_mse: 1.5936 - val_mape: 333.7762 - val_smape: 132.6933 - 775ms/epoch - 129ms/step
Epoch 5/200
6/6 - 1s - loss: 137.8001 - mae: 0.9737 - mse: 1.5317 - mape: 322.5908 - smape: 

# From here on out, it is assumed that best_model is found

In [127]:
m = tuner.hypermodel.build(best_hps)
m.load_weights(checkpoint_path)
# m.evaluate(window.test)

In [128]:
# window.test

# # val_performance['1'] = m.evaluate(w.val)
# for i in range(6):

#     label = label_columns[i]
#     print(label)
#     # performance['1'] = m.evaluate(w.test)
#     window.plot(m, plot_col=label, max_subplots=7)


In [129]:
inputs, labels, predictions, weights, mcds = [], [], [], [], []
for x, y in window.test.take(40):
    inputs.append(x)
    lab = y
    lab = window.label_postprocessor.reverse(lab)
    labels.append(lab)
    
    pred = m(x)
    pred = window.label_postprocessor.reverse(pred)
    predictions.append(pred)
    
    weight = m.attention_layer(x, return_weights=True)[1]
    weights.append(weight)
    
    mcd = results.monte_carlo_dropout(x, m, 100, window.label_postprocessor.reverse, return_weight=False)
    mcds.append(mcd)
    weights.append(weight)
    
inputs = tf.concat(inputs, axis=0)
labels = tf.concat(labels, axis=0)
weights = tf.concat(weights, axis=0)
weights = tf.reduce_mean(weights, axis=0)
predictions = tf.concat(predictions, axis=0)
mcds = tf.concat(mcds, axis=1)

In [130]:
np.save(model_path / f"{product}_inputs_{data_type}", inputs.numpy())
np.save(model_path / f"{product}_labels_{data_type}", labels.numpy())
np.save(model_path / f"{product}_weights_{data_type}", weights.numpy())
np.save(model_path / f"{product}_predictions_{data_type}", predictions.numpy())
np.save(model_path / f"{product}_mcd_predictions_{data_type}", mcds.numpy())
